In [72]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from matplotlib import pyplot as plt
import seaborn as sns
import vccfcoords
import math
import localization as lx
import os, sys

In [79]:
def sim_coordinates(n, ndim, min=0, max=1):

    val_range = max - min
    generated_coordinates = np.random.rand(n, ndim) * val_range + min
    #coord_dict = {x:generated_coordinates[x] for x in list(range(n))}
    
    return generated_coordinates

def get_euclid_dist(pt1, pt2):

    return distance.euclidean(pt1, pt2)

def get_closest_points(coords, target, n):

    distances = distance.cdist([target], coords, metric='euclidean')[0]

    idx = np.argsort(distances)[:n]

    return idx
    
def dict_to_array(coord_dict):

    return np.vstack(list(coord_dict.values()))


def plot_coordinates(coords, ndim):

    assert(ndim in [2,3])

    #coords = dict_to_array(coord_dict)

    if ndim == 2:
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]

        plt.scatter(x_vals, y_vals)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()

        return
    
    else:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Generate the values
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]
        z_vals = coords[:, 2:3]

        # Plot the values
        ax.scatter(x_vals, y_vals, z_vals, c = 'b', marker='o')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')

        plt.show()

        return

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def estimate_coord(coords, distances):
    
    dim = coords.shape[1]
    dim_str = str(dim) + "D"

    state = lx.Project(mode=dim_str,solver='LSE')

    for coord_idx in range(len(coords)):
        state.add_anchor(str(coord_idx), tuple(coords[coord_idx]))
    
    unknown_cell, unkown_cell_label= state.add_target()

    for dist_idx in range(len(distances)):
        unknown_cell.add_measure(str(dist_idx),distances[dist_idx])

    with HiddenPrints():
        state.solve()

    computed_coord = None

    if dim == 2:
        computed_coord = (unknown_cell.loc.x, unknown_cell.loc.y)
    else:
        computed_coord = (unknown_cell.loc.x, unknown_cell.loc.y, unknown_cell.loc.z)

    return computed_coord


def estimate_coords(reference_coords, closest_cells):

    



In [84]:
estimate_coord(np.array([[0,1,1],[0,0,1],[0,0.5,0.5]]), [0.5,0.5,0.5])

(2.6398153217214215e-09, 0.49999997891568426, 0.9999999926850489)

In [51]:
#Generating Cell Coordinates
EC_coords = sim_coordinates(1000, 3, -1, 1)
other_cell_coords = sim_coordinates(10000, 3, -1, 1)

#Computing Closest Neighbors to Each Cell
closest_EC_cells_3 = {x:get_closest_points(EC_coords,other_cell_coords[x], 3) for x in list(range(len(other_cell_coords)))}

In [52]:
closest_EC_cells_3

{0: array([261, 408,  97]),
 1: array([ 21, 779, 886]),
 2: array([535,  33, 994]),
 3: array([562, 164, 449]),
 4: array([882, 466, 963]),
 5: array([ 65,  96, 951]),
 6: array([894, 167, 612]),
 7: array([827, 621,  97]),
 8: array([238, 826,  30]),
 9: array([476, 294, 530]),
 10: array([326, 821, 704]),
 11: array([188, 530, 476]),
 12: array([416, 203, 803]),
 13: array([ 85, 380, 441]),
 14: array([903, 606, 976]),
 15: array([935, 100, 690]),
 16: array([509, 565, 733]),
 17: array([630, 577, 706]),
 18: array([930, 167, 511]),
 19: array([ 20, 552, 365]),
 20: array([683, 696, 317]),
 21: array([315, 839, 907]),
 22: array([431,  56, 888]),
 23: array([501, 682, 266]),
 24: array([485, 587, 949]),
 25: array([477, 295, 672]),
 26: array([808, 346, 700]),
 27: array([981, 384,  72]),
 28: array([611, 525, 575]),
 29: array([369, 888, 160]),
 30: array([582, 702, 329]),
 31: array([305, 159,  25]),
 32: array([599, 213, 712]),
 33: array([ 76, 911, 668]),
 34: array([256, 450, 73

In [55]:
other_cell_coords[0]

array([ 0.09814272, -0.03299581,  0.59333181])

In [57]:
EC_coords[closest_EC_cells_3[0]]

array([[-0.40882984,  0.67813756, -0.94476455],
       [-0.62338338,  0.6155872 , -0.92901035],
       [-0.50300839,  0.75957002, -0.81990308],
       [-0.63502526,  0.71578079, -0.87368287]])

In [47]:
import localization as lx

P=lx.Project(mode='2D',solver='LSE')
P.add_anchor('anchorA',(0,1))
P.add_anchor('anchorB',(0,1))

t,label=P.add_target()


t.add_measure('anchorA',50)
t.add_measure('anchorB',0)


P.solve()

# Then the target location is:

print(t.loc)

LSE Geolocating...
p(17.677667328562006,18.677667328562006,0.0)


In [48]:
t

In [60]:
b = t.loc

In [63]:
b.z

0.0